In [73]:
import pandas as pd
import random
import dask.dataframe as dd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [74]:
# Load the data
sample_df = dd.read_csv("../data/1000_15_2021.csv").compute()
sample_df.head()

,Unnamed: 0,id,timestamp,value_kwh,year
0,1576738,e255670711f8683117c02494fee87c3e,2021-01-20T08:45:00.000Z,0.141,2021
1,1576739,24e56bc28aff45dd646668bd55c13715,2021-01-20T03:30:00.000Z,0.208,2021
2,1576740,968da760f41ab2ee25239a174561876e,2021-01-19T23:00:00.000Z,0.372,2021
3,1576741,30a143bad61613b4bac4adfe013364a8,2021-01-20T12:00:00.000Z,0.003,2021
4,1576742,5caf126c212c16123ead53bd9bf8394f,2021-01-28T15:15:00.000Z,0.020,2021


In [75]:
# Add month column
sample_df["timestamp"] = pd.to_datetime(sample_df["timestamp"])
sample_df["year_month"] = sample_df["timestamp"].dt.to_period("M")
sample_df.head()

C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\135626291.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  sample_df["year_month"] = sample_df["timestamp"].dt.to_period("M")


,Unnamed: 0,id,timestamp,value_kwh,year,year_month
0,1576738,e255670711f8683117c02494fee87c3e,2021-01-20 08:45:00+00:00,0.141,2021,2021-01
1,1576739,24e56bc28aff45dd646668bd55c13715,2021-01-20 03:30:00+00:00,0.208,2021,2021-01
2,1576740,968da760f41ab2ee25239a174561876e,2021-01-19 23:00:00+00:00,0.372,2021,2021-01
3,1576741,30a143bad61613b4bac4adfe013364a8,2021-01-20 12:00:00+00:00,0.003,2021,2021-01
4,1576742,5caf126c212c16123ead53bd9bf8394f,2021-01-28 15:15:00+00:00,0.020,2021,2021-01


In [76]:
def compute_monthly_features(dfgr: pd.DataFrame):
    features = {}

    # Basic features
    features["load_mean"] = dfgr["value_kwh"].mean()
    features["load_std"] = dfgr["value_kwh"].std() if features["load_mean"] != 0 else 0
    features["load_std"] = features["load_std"] / features["load_mean"] if features["load_mean"] != 0 else 0
    features["load_min"] = dfgr["value_kwh"].min() if features["load_mean"] != 0 else 0
    features["load_min"] = features["load_min"] / features["load_mean"] if features["load_mean"] != 0 else 0
    features["load_max"] = dfgr["value_kwh"].max() if features["load_mean"] != 0 else 0
    features["load_max"] = features["load_max"] / features["load_mean"] if features["load_mean"] != 0 else 0

    features["load_daily_mean_std"] = dfgr.set_index("timestamp")["value_kwh"].resample("D").mean().std() if features["load_mean"] != 0 else 0
    features["load_daily_mean_std"] = features["load_daily_mean_std"] / features["load_mean"] if features["load_mean"] != 0 else 0
    for weekday in range(7):
        features[f"load_weekday_{weekday}_mean"] = dfgr[dfgr["timestamp"].dt.weekday == weekday]["value_kwh"].mean() if features["load_mean"] != 0 else 0
        features[f"load_weekday_{weekday}_mean"] = features[f"load_weekday_{weekday}_mean"] / features["load_mean"] if features["load_mean"] != 0 else 0
        features[f"load_weekday_{weekday}_std"] = dfgr[dfgr["timestamp"].dt.weekday == weekday]["value_kwh"].std() if features["load_mean"] != 0 else 0
        features[f"load_weekday_{weekday}_std"] = features[f"load_weekday_{weekday}_std"] / features["load_mean"] if features["load_mean"] != 0 else 0
    
    dfgr["weekofyear"] = dfgr["timestamp"].dt.isocalendar().week
    features["load_mean_weekly_max"] = dfgr.groupby("weekofyear")["value_kwh"].max().mean() if features["load_mean"] != 0 else 0
    features["load_mean_weekly_max"] = features["load_mean_weekly_max"] / features["load_mean"] if features["load_mean"] != 0 else 0
    features["load_mean_weekly_min"] = dfgr.groupby("weekofyear")["value_kwh"].min().mean() if features["load_mean"] != 0 else 0
    features["load_mean_weekly_min"] = features["load_mean_weekly_min"] / features["load_mean"] if features["load_mean"] != 0 else 0

    dfgr["timestamp"] = pd.to_datetime(dfgr["timestamp"])
    tmp = dfgr.sort_values("timestamp").set_index("timestamp").copy()
    tmp["mean"] = tmp["value_kwh"].rolling(3*4, center=True).mean()
    tmp["std"] = tmp["value_kwh"].rolling(3*4, center=True).std()
    features["count_outliers_high"] = (tmp["value_kwh"] > tmp["mean"] + 2.5*tmp["std"]).sum()
    features["count_outliers_low"] = (tmp["value_kwh"] < tmp["mean"] - 1.5*tmp["std"]).sum()

    return pd.Series(features)

In [78]:
def get_week_load_vectors_per_id(tmp: pd.DataFrame):

    def _apply(dfgr: pd.DataFrame):
        dfgr["timestamp"] = pd.to_datetime(dfgr["timestamp"])
        dfgr.sort_values("timestamp", inplace=True)
        dfgr = dfgr.set_index("timestamp")[["value_kwh"]].resample("15min").interpolate()
        dfgr["value_kwh"] = dfgr["value_kwh"].rolling(4).mean().bfill().ffill()
        dfgr["value_kwh"] = (dfgr["value_kwh"] - dfgr["value_kwh"].min()) / max(1e-12, (dfgr["value_kwh"].max() - dfgr["value_kwh"].min()))
        data = {f"point_{i}": [dfgr["value_kwh"].values[i]] for i in range(len(dfgr))}
        return pd.DataFrame(data)
    
    tmp = tmp.groupby("id").apply(_apply)

    tmp = tmp.apply(lambda x: x.ffill().bfill(), axis=0)

    return tmp.droplevel(1)

def get_embeddings(tmp: pd.DataFrame):
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(tmp.values)

    results = pd.DataFrame(tsne_results, columns=["x", "y"], index=tmp.index)
    
    return results

In [79]:
sample_df["weekofyear"] = sample_df["timestamp"].dt.isocalendar().week

In [80]:
tmps = []
for year_month in sorted(sample_df["year_month"].unique())[:4]:
    print(f"Processing {year_month}")
    tmps2 = []
    for i, week in enumerate(sample_df[sample_df["year_month"] == year_month]["weekofyear"].unique()[:4]):
        tmp = sample_df[(sample_df["year_month"] == year_month) & (sample_df["weekofyear"] == week)].copy()
        tmp = get_week_load_vectors_per_id(tmp)
        tmp = get_embeddings(tmp)
        tmp.columns = [f"{col}_{i}" for col in tmp.columns]
        tmps2.append(tmp)
    tmp = tmps2[0]
    for tmp2 in tmps2[1:]:
        tmp = tmp.merge(tmp2, left_index=True, right_index=True)
    tmp["year_month"] = year_month
    tmps.append(tmp)
    print(f"Done processing {year_month}")

Processing 2021-01


C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFr

Done processing 2021-01
Processing 2021-02


C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFr

Done processing 2021-02
Processing 2021-03


C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFr

Done processing 2021-03
Processing 2021-04


C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("id").apply(_apply)
C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3857518526.py:12: DeprecationWarning: DataFr

Done processing 2021-04


In [81]:
tsne_embeddings = pd.concat(tmps, axis=0)

In [82]:
tsne_embeddings

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,year_month
id,,,,,,,,,
0057bc119b36b4d0dcfef05e3f60163e,7.050730,12.585196,-11.554958,-10.121217,4.546989,-15.588981,14.557451,-0.533628,2021-01
0094af6aa5e6e288830c31902c662df7,25.054365,5.191094,12.971921,-13.654313,19.535120,-12.726646,27.621145,1.784444,2021-01
00a6f112d975507b85d6e1de1218f492,-15.953628,-3.417279,-10.041182,9.883196,-15.049474,8.924443,-16.105347,0.705636,2021-01
00d3db5764921ceb4175205c7244a658,-12.339955,16.199697,-8.002400,0.543767,-7.966089,-0.823009,-2.174746,-15.524407,2021-01
00e599b0ee0efa0db33130d7d3c3e112,-1.397745,8.842761,-13.625495,-5.757251,-17.010511,-3.088088,4.259972,-8.137360,2021-01
...,...,...,...,...,...,...,...,...,...
ff05b289104b79232e67fbf0ca735e68,6.212522,10.467321,5.075689,8.339727,4.884587,1.292980,4.536036,7.666751,2021-04
ff644c59d5d411d23e7bbd6e511d8754,-13.124590,6.917380,-1.182650,6.724320,-12.624637,-20.523670,-1.573581,-7.811801,2021-04
ffa17decabd2683d6cc0054eb3b2541c,-9.667359,15.240415,4.067368,14.047717,19.064936,-4.124770,14.651155,-8.243841,2021-04


In [83]:
tsne_embeddings = tsne_embeddings.reset_index()

In [84]:
# Step 1: Group by 'id' and extract features
monthly_features_df = sample_df.groupby(["id", "year_month"]).apply(compute_monthly_features)

monthly_features_df = monthly_features_df.reset_index()
monthly_features_df.head()

C:\Users\swiss_sdi\AppData\Local\Temp\ipykernel_29116\3560796465.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_features_df = sample_df.groupby(["id", "year_month"]).apply(compute_monthly_features)


,id,year_month,load_mean,load_std,load_min,load_max,load_daily_mean_std,load_weekday_0_mean,load_weekday_0_std,load_weekday_1_mean,...,load_weekday_4_mean,load_weekday_4_std,load_weekday_5_mean,load_weekday_5_std,load_weekday_6_mean,load_weekday_6_std,load_mean_weekly_max,load_mean_weekly_min,count_outliers_high,count_outliers_low
0,0057bc119b36b4d0dcfef05e3f60163e,2021-01,0.184362,0.760767,0.347143,7.789030,0.160007,0.892892,0.667940,1.023449,...,1.017067,0.616293,1.059446,0.851211,0.990195,0.909063,6.318010,0.357992,42.0,35.0
1,0057bc119b36b4d0dcfef05e3f60163e,2021-02,0.180484,0.736772,0.365684,6.432713,0.180156,1.018296,0.702099,1.056715,...,0.850369,0.570779,1.089829,0.881470,0.950886,0.791944,5.320424,0.393387,27.0,35.0
2,0057bc119b36b4d0dcfef05e3f60163e,2021-03,0.164052,0.730755,0.390121,5.723808,0.132283,0.967543,0.724380,1.112090,...,0.903915,0.571388,0.915142,0.708501,0.923290,0.745884,5.319057,0.407189,38.0,33.0
3,0057bc119b36b4d0dcfef05e3f60163e,2021-04,0.163328,0.726846,0.391850,5.957351,0.139459,0.964529,0.752873,1.171416,...,0.948409,0.675389,0.938948,0.683095,0.911701,0.653104,5.637748,0.412667,47.0,33.0
4,0057bc119b36b4d0dcfef05e3f60163e,2021-05,0.169429,0.721692,0.377740,8.097801,0.157548,1.067345,0.764517,1.027058,...,0.952091,0.579828,0.931572,0.715739,1.108253,0.917884,5.894318,0.419055,36.0,59.0


In [85]:
monthly_features_df_copy = monthly_features_df.copy()
monthly_features_df_copy = monthly_features_df_copy.merge(tsne_embeddings, on=["id", "year_month"])

In [86]:
monthly_features_df_copy

,id,year_month,load_mean,load_std,load_min,load_max,load_daily_mean_std,load_weekday_0_mean,load_weekday_0_std,load_weekday_1_mean,...,count_outliers_high,count_outliers_low,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3
0,0057bc119b36b4d0dcfef05e3f60163e,2021-01,0.184362,0.760767,0.347143,7.789030,0.160007,0.892892,0.667940,1.023449,...,42.0,35.0,7.050730,12.585196,-11.554958,-10.121217,4.546989,-15.588981,14.557451,-0.533628
1,0057bc119b36b4d0dcfef05e3f60163e,2021-02,0.180484,0.736772,0.365684,6.432713,0.180156,1.018296,0.702099,1.056715,...,27.0,35.0,9.240581,0.753598,-4.580644,0.577361,12.543537,-0.462163,4.420297,-16.840569
2,0057bc119b36b4d0dcfef05e3f60163e,2021-03,0.164052,0.730755,0.390121,5.723808,0.132283,0.967543,0.724380,1.112090,...,38.0,33.0,8.331800,-5.526264,-1.779780,-4.932122,2.736141,-0.228069,2.818588,8.047970
3,0057bc119b36b4d0dcfef05e3f60163e,2021-04,0.163328,0.726846,0.391850,5.957351,0.139459,0.964529,0.752873,1.171416,...,47.0,33.0,-1.471693,9.671637,5.979890,9.927336,5.950096,2.105975,11.946733,-0.198124
4,0094af6aa5e6e288830c31902c662df7,2021-01,0.018490,0.672532,0.000000,2.163347,0.309863,0.938678,0.629583,0.947524,...,39.0,86.0,25.054365,5.191094,12.971921,-13.654313,19.535120,-12.726646,27.621145,1.784444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,ffcbe4ad5d73c72645aafb124833705c,2021-04,0.061986,1.127588,0.064531,4.678471,0.948671,1.136677,1.216793,1.220494,...,27.0,47.0,7.277055,2.045863,27.765862,3.729692,8.450747,19.688030,16.371431,3.277364
3996,fff08a4ca55dff8b1b402616dd131178,2021-01,0.076483,0.624163,0.483765,4.550008,0.093783,1.004340,0.581089,0.942947,...,57.0,4.0,4.979517,9.918440,8.807549,-13.531625,9.573972,-11.302052,6.940487,-15.677040
3997,fff08a4ca55dff8b1b402616dd131178,2021-02,0.087202,0.683693,0.424304,4.862289,0.092179,1.047722,0.752205,0.993322,...,54.0,2.0,10.974630,7.195539,-11.455105,0.242695,14.952992,-0.806287,11.282990,-5.809941
3998,fff08a4ca55dff8b1b402616dd131178,2021-03,0.088602,0.696081,0.428883,8.216504,0.119440,1.056868,0.890008,1.077498,...,55.0,4.0,10.365964,-6.129517,13.707472,-9.670972,18.726225,3.819096,-2.498168,12.477772


In [87]:
# Save the data as csv
monthly_features_df_copy.to_csv("../data/1000_15_2021_features_3.csv", index=False)